In [ ]:
#导入库
import numpy as np          # 数值计算库，处理数组
import xarray as xr         # 处理NetCDF格式数据
import rasterio             # 读取地理栅格数据（如.tif）
from scipy.interpolate import griddata  # 空间插值工具
from datetime import datetime          # 处理时间信息

In [ ]:
#读取极端降水数据
extreme_pr_path = 'D:\\CMIP6extreme\\00 - CN051-2022\\extreme_pr\\extreme_pr_index_1961_2022.nc'
extreme_pr_ds = xr.open_dataset(extreme_pr_path)  # 打开NetCDF文件

lat = extreme_pr_ds['lat'].values  # 获取纬度数组
lon = extreme_pr_ds['lon'].values  # 获取经度数组

In [ ]:
#定义中国区域范围，一个包含中国的矩形
lat_min, lat_max = 15, 55  # 纬度范围
lon_min, lon_max = 70, 140 # 经度范围

In [ ]:
#人口数据读取函数
def read_and_clip_population_data(year):
    # 构建人口数据文件路径（LandScan数据）
    pop_path = f'D:\\CMIP6extreme\\00 - CN051-2022\\extreme_pr\\人口暴露\\landscan-global-{year}-colorized.tif'
    
    with rasterio.open(pop_path) as src:  # 打开栅格文件
        pop_data = src.read(1, masked=True)  # 读取第一个波段，处理缺失值
        transform = src.transform  # 获取地理变换参数
        
        # 生成经纬度网格
        pop_lon, pop_lat = np.meshgrid(
            np.arange(pop_data.shape[1]) * transform[0] + transform[2],
            np.arange(pop_data.shape[0]) * transform[4] + transform[5]
        )

        # 裁剪到中国范围
        mask = (pop_lat >= lat_min) & (pop_lat <= lat_max) & (pop_lon >= lon_min) & (pop_lon <= lon_max)
        pop_lat = pop_lat[mask]  # 筛选纬度
        pop_lon = pop_lon[mask]  # 筛选经度
        pop_data = pop_data[mask]  # 筛选人口数据
    
    return pop_lat, pop_lon, pop_data  # 返回筛选后的经纬度和人口数据

#地理变换参数（transform）包含6个参数的仿射变换矩阵
# | a  b  c |
# | d  e  f |
# | 0  0  1 |

# a（transform[0]）：东西方向像素分辨率（单位：度/像素）
# e（transform[4]）：南北方向像素分辨率（通常为负值，因为纬度向下递减）
# c（transform[2]）：左上角像素中心点的经度
# f（transform[5]）：左上角像素中心点的纬度

# 生成列坐标（经度）    
# np.arange(pop_data.shape[1]) * transform[0] + transform[2]
# pop_data.shape[1]：栅格的列数（东西方向像素数量）
# np.arange(pop_data.shape[1])：生成列索引数组 [0, 1, 2, ..., cols-1]
# 计算逻辑：
# 每列的经度 = 列索引 × 经度分辨率 + 左上角起始经度

# 生成行坐标（纬度）
# np.arange(pop_data.shape[0]) * transform[4] + transform[5]
# pop_data.shape[0]：栅格的行数（南北方向像素数量）
# np.arange(pop_data.shape[0])：生成行索引数组 [0, 1, 2, ..., rows-1]
# 注意：transform[4]通常为负数，因为纬度值从上到下递减
# 计算逻辑：
# 每行的纬度 = 行索引 × 纬度分辨率 + 左上角起始纬度

In [ ]:
#插值函数
def interpolate_population_to_precip_grid(pop_lat, pop_lon, pop_data, precip_lat, precip_lon):
    # 将人口数据插值到极端降水数据的网格上
    points = np.array([pop_lon.flatten(), pop_lat.flatten()]).T  # 原始点坐标， .flatten()将数据平整为一维
    grid_x, grid_y = np.meshgrid(precip_lon, precip_lat)  # 目标网格
    
    # 执行插值（双线性插值）
    grid_data = griddata(
        points,                # 原始数据点坐标
        pop_data.flatten(),    # 原始数据值
        (grid_x, grid_y),      # 目标网格
        method='linear'        # 插值方法：线性
    )
    
    return grid_data  # 返回网格化的人口数据


In [ ]:
#初始化结果容器
# 定义计算年份范围（2000-2022）
years = range(2000, 2023)

# 定义要计算的极端降水指标
exposure_indices = ['max_1_day_precip', 'max_5_day_precip', 'wetdays_20mm', 
                   'wetdays_10mm', 'max_consec_wet_days', 'max_consec_dry_days', 
                   'daily_pr_intensity', 'prcptot']

# 创建存储结果的字典，每个指标对应三维数组（年份×纬度×经度）
pop_exposure = {index: np.zeros((len(years), len(lat), len(lon)), dtype=np.float32) 
               for index in exposure_indices}


In [ ]:
#主处理循环
for idx, year in enumerate(years):
    print(f'Processing year: {year}')
    
    # 读取并裁剪人口数据
    pop_lat, pop_lon, pop_data = read_and_clip_population_data(year)
    
    # 将人口数据插值到降水数据网格
    interpolated_pop = interpolate_population_to_precip_grid(pop_lat, pop_lon, pop_data, lat, lon)
    
    # 对每个极端降水指数进行计算
    for index in exposure_indices:
        # 读取当年的降水数据
        precip_data = extreme_pr_ds[index].sel(time=str(year)).values.astype(np.float32)
        
        # 计算暴露量：降水指数 × 对应位置的人口
        pop_exposure[index][idx, :, :] = precip_data * interpolated_pop


In [ ]:
#保存结果
output_filename = 'population_exposure_2000_2022.nc'

# 创建Xarray数据集
ds = xr.Dataset(
    {index: (('time', 'lat', 'lon'), pop_exposure[index]) for index in exposure_indices},
    coords={
        'time': years,  # 时间坐标
        'lat': lat,     # 纬度坐标
        'lon': lon      # 经度坐标
    }
)

# 设置元数据
ds.time.attrs['units'] = 'year'
ds.lat.attrs['units'] = 'degrees_north'
ds.lon.attrs['units'] = 'degrees_east'
for index in exposure_indices:
    ds[index].attrs['units'] = 'people_exposed'

# 全局属性
ds.attrs['title'] = 'Population Exposure to Extreme Precipitation (2000-2022)'
ds.attrs['history'] = f'Created {datetime.utcnow().isoformat()}'

# 保存为NetCDF文件
ds.to_netcdf(output_filename)
print(f'Population exposure data has been saved to {output_filename}')
